<a href="https://colab.research.google.com/github/floflokie/TP_BigData/blob/main/Chapitre_3_Spark_SQL_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP : Spark SQL

Imagine you're analyzing sales data for a retail company. You need to answer questions like: "What is the average order value per customer?" and "Which products are most popular in each region?"

By using Spark SQL to query structured data in a DataFrame, you can efficiently analyze large datasets that would be cumbersome with traditional SQL.

In [1]:
import pandas as pd
import numpy as np

# Generate a fake dataset
data = {
    'transaction_id': np.arange(1, 500001),
    'product_id': np.random.randint(1, 1000, 500000),
    'product_category': np.random.choice(['Electronics', 'Clothing', 'Books'], 500000),
    'region': np.random.choice(['North', 'South', 'East', 'West'], 500000),
    'date': pd.date_range(start="2021-01-01", periods=500000, freq='T').to_numpy(),
    'units_sold': np.random.randint(1, 10, 500000),
    'price_per_unit': np.round(np.random.uniform(5.0, 500.0, 500000), 2)
}
df = pd.DataFrame(data)

<ipython-input-1-6053c13f21be>:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'date': pd.date_range(start="2021-01-01", periods=500000, freq='T').to_numpy(),


TODO: convert to Spark Dataframe

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Sales Analysis").getOrCreate()

In [3]:
sales_df = spark.createDataFrame(df)

TODO: Register DataFrame as a SQL temporary view to be able to write queries


In [4]:
sales_df.registerTempTable("sales")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
spark.sql("""
    SELECT region, product_category, SUM(units_sold * price_per_unit) AS total_sales
    FROM sales
    GROUP BY region, product_category
    ORDER BY total_sales DESC
""").show()


+------+----------------+--------------------+
|region|product_category|         total_sales|
+------+----------------+--------------------+
|  West|           Books| 5.293862394999991E7|
| North|           Books|5.2936714270000115E7|
|  East|           Books|5.2868406039999954E7|
| North|        Clothing| 5.269190509000005E7|
|  East|        Clothing| 5.262250680999987E7|
|  West|     Electronics|5.2618212639999926E7|
| South|     Electronics| 5.259139255999991E7|
| South|           Books| 5.256330692000005E7|
|  West|        Clothing| 5.240502004999998E7|
|  East|     Electronics| 5.240054752000001E7|
| North|     Electronics| 5.204253912999967E7|
| South|        Clothing|5.2028716859999925E7|
+------+----------------+--------------------+



TODO : write and execute query to get the top 5 products by sales

In [6]:
spark.sql("""
    SELECT product_id, SUM(units_sold * price_per_unit) AS total_sales
    FROM sales
    GROUP BY product_id
    ORDER BY total_sales DESC
    LIMIT 5
""").show()

+----------+-----------------+
|product_id|      total_sales|
+----------+-----------------+
|       999|        737270.54|
|       825|734966.1000000001|
|        81|733931.3700000001|
|       186|        733395.74|
|       522|732284.5000000002|
+----------+-----------------+

